In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
filename ='supply_chain_data'

In [18]:
def ConvertDataFormat(formatType:str):
    import pandas as pd

    try:

     
        scmdf =pd.read_csv("..\\"+filename+".csv",header=0)

        #display(scmdf)


        if formatType == 'json':
            scmdf.to_json(filename+".json",orient="records")
        elif formatType == 'parquet':
            scmdf.to_parquet(filename,compression='snappy')

    except Exception as error:
        print('Error Occurred' , error)

In [15]:
import pandas as pd
import json

#Step 0: Convert the csv data into given format
ConvertDataFormat('json')

# Step 1: Read the JSON file
file_path = filename+'.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Step 2: Load JSON data into a DataFrame
df = pd.json_normalize(data)

# Step 3: Data Cleansing and Massaging

"""df.rename(columns={
    'productID': 'Product_ID',
    'productName': 'Product_Name',
    'supplierName': 'Supplier_Name',
    'orderDate': 'Order_Date',
    'deliveryDate': 'Delivery_Date',
    'quantity': 'Quantity',
    'unitPrice': 'Unit_Price',
    'totalPrice': 'Total_Price'
}, inplace=True)"""



# Handling missing values
# Fill missing values for 'Quantity' and 'Unit_Price' with 0
df['Quantity'].fillna(0, inplace=True)
df['Price'].fillna(0, inplace=True)

# Fill missing values for 'Supplier_Name' with 'Unknown Supplier'
df['Supplier_Name'].fillna('Unknown Supplier', inplace=True)

# Converting data types
# Convert 'Quantity' and 'Unit_Price' to numeric
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Calculating Total Price if missing
df['Total_Price'] = df['Quantity'] * df['Price']

# Removing duplicates
df.drop_duplicates(inplace=True)

# Filtering data
# Filter out records where 'Quantity' is zero
df = df[df['Quantity'] > 0]

# Parsing dates
#df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%Y-%m-%d', errors='coerce')
#df['Delivery_Date'] = pd.to_datetime(df['Delivery_Date'], format='%Y-%m-%d', errors='coerce')

# Calculating lead time (difference between order and delivery dates)
#df['Lead_Time'] = (df['Delivery_Date'] - df['Order_Date']).dt.days

# Step 4: Save the cleansed and massaged DataFrame to a new CSV file
output_file_path = 'preprocessed_supply_chain_data.csv'
df.to_csv(output_file_path, index=False)

print("Data has been cleaned and saved to:", output_file_path)


Data has been cleaned and saved to: preprocessed_supply_chain_data.csv


In [19]:
ConvertDataFormat('parquet')

scmdf = pd.read_parquet(filename+'.parquet')

display(scmdf)



FileNotFoundError: [Errno 2] No such file or directory: 'supply_chain_data.snappy.parquet'